## Dataset Exploration:


In [48]:
#!pip install duckdb
#!pip3 install pyspark
#!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"
#!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS pyspark_postgres;'
#!sudo -u postgres psql -U postgres -c 'CREATE DATABASE pyspark_postgres;'
#!wget -O "postgresql.jar" "https://repo1.maven.org/maven2/org/postgresql/postgresql/42.7.3/postgresql-42.7.3.jar"
#!wget -O "duckdb.jar" "https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar"


In [12]:
## Imports
import duckdb
import pyspark
from pyspark.sql import SparkSession
from pprint import pprint

## Connection to formatted database
jdbc_url = 'jdbc:duckdb:./../data/formatted_zone/barcelona.db'
driver = "org.duckdb.DuckDBDriver"

# SparkSession inicialitzation
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .appName("DataExploration") \
    .getOrCreate()

#### Criminal Dataset


In [13]:
df = spark.read \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("driver", driver) \
  .option("query", "SELECT * FROM df_criminal_dataset") \
  .load()

df.show()

+----+-------+--------+--------------------+-----------------------+---------+-----------------+--------------------+-----------------------+--------------------+--------------------+------------------------+-------------------------+--------------------+-------------------------+---------------+
| any|n_m_mes| nom_mes|    regi_policial_rp|rea_b_sica_policial_abp|prov_ncia|          comarca|            municipi|tipus_de_lloc_dels_fets|     canal_dels_fets|        tipus_de_fet|t_tol_del_fet_codi_penal|tipus_de_fet_codi_penal_o|            mbit_fet|nombre_fets_o_infraccions|nombre_v_ctimes|
+----+-------+--------+--------------------+-----------------------+---------+-----------------+--------------------+-----------------------+--------------------+--------------------+------------------------+-------------------------+--------------------+-------------------------+---------------+
|2021|     12|Desembre|RP  Metropolitana...|           ABP Badalona|Barcelona|       Barcelonès|          

In [ ]:
spark.stop()